In [71]:
#importing packages
from collections import Counter, defaultdict
import math
import numpy as np
import os
import pandas as pd
import re
from scipy.sparse import csr_matrix
#import urllib.request
#import zipfile

# Use nltk for valid words
import nltk
# Need to make hash 'dictionaries' from nltk for fast processing
import collections as co
import csv


In [25]:
# Read the input
d = pd.read_csv("F:\IITC\ADM\Extra\consumer_complaints.csv") # the consumer dataset is now a Pandas DataFrame
# Only interested in data with consumer complaints
d = d[d['Consumer complaint narrative'].notnull()]

C:\Users\Swapnil\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [26]:
print(d['Consumer complaint narrative'])

56777     Received Capital One charge card offer XXXX. A...
56834     I do n't know how they got my cell number. I t...
56884     I 'm a longtime member of Charter One Bank/RBS...
56894     After looking at my credit report, I saw a col...
56898     I received a call from a XXXX XXXX from XXXX @...
56910     Was not contacted 4 years later about some pri...
56911     Collection Consultants is reporting a collecti...
56914     I had my purse stolen in 2007. They never foun...
56917     I attempted to apply for a Discover Card Onlin...
56918     Continued attempts by XXXX XXXX XXXX to collec...
56920     This is a continuation of a previous issue wit...
56928     Going through a divorce, my ex and I were unab...
56930     I am assisted with my mortgage through an agen...
56939     i submitted a payment on XXXX XXXX 2015 for my...
56940     I recieved a notice from Midland Credit Manage...
56956     XXXX Card services was bought out by Capital O...
56965     I was reported late by Discove

In [27]:
print(d[1:5])

      Date received           Product                            Sub-product  \
56834     3/23/2015   Debt collection  Other (i.e. phone, health club, etc.)   
56884     3/23/2015       Credit card                                    NaN   
56894     3/23/2015  Credit reporting                                    NaN   
56898     3/23/2015   Debt collection                            Payday loan   

                                        Issue  \
56834     Improper contact or sharing of info   
56884                                 Rewards   
56894  Incorrect information on credit report   
56898     Improper contact or sharing of info   

                                   Sub-issue  \
56834      Contacted me after I asked not to   
56884                                    NaN   
56894                Information is not mine   
56898  Talked to a third party about my debt   

                            Consumer complaint narrative  \
56834  I do n't know how they got my cell number. I 

In [28]:
def tokenize_string(my_string):
    """ DONE. You should use this in your tokenize function.
    """
    #return re.findall('[\w\-]+', my_string.lower())
    #\W -> Matches any non-alphanumeric character; 
    #this is equivalent to the class [^a-zA-Z0-9_]. 
      
    # preprocessing ->
    # remove numbers    
    word_list = re.sub('[^a-zA-Z]+', ' ', my_string.lower()).split()
    
   
    # remove stop words
    stopWords = co.Counter( nltk.corpus.stopwords.words() )

    #print(stopWords)
    
    clean_word_list = [x for x in word_list if x not in stopWords]
    
    #print(clean_word_list)
    
    
    # remove small and large words
   
    word_list = [x for x in clean_word_list if len(x) > 4 and len(x) < 15 ]
    
    
    # remove xxxx
    
    clean_word_list = [x for x in word_list if "xxxx" not in x ]
   
    # remove too low and too high frequency words -> we can't calculate here

    
    
    print(clean_word_list)
    
    return(clean_word_list)

In [29]:
def tokenize(data):
    """
    Append a new column to the movies DataFrame with header 'tokens'.
    This will contain a list of strings, one per token, extracted
    from the 'genre' field of each movie. Use the tokenize_string method above.
    Note: you may modify the movies parameter directly; no need to make
    a new copy.
    Params:
      movies...The movies DataFrame
    Returns:
      The movies DataFrame, augmented to include a new column called 'tokens'.
    >>> movies = pd.DataFrame([[123, 'Horror|Romance'], [456, 'Sci-Fi']], columns=['movieId', 'genres'])
    >>> movies = tokenize(movies)
    >>> movies['tokens'].tolist()
    [['horror', 'romance'], ['sci-fi']]
    """
    ###TODO
    
    # step 1 -> do tokenize_string for each row in movies['genres']

    all_words = []
    for row in data['Consumer complaint narrative']:
        #genre_list = re.sub(r'[||)|(]', r' ',row.lower()).split()
        word_list = tokenize_string(row)
        #print(word_list)
        #print(len(genre_list))
        all_words.append(word_list)
    
    # step 2 -> add column tokens in movies
    array = np.array(all_words)
    
    #print(array[:5])
    #print('#list = ',len(array))
    
    new_data = data.assign(tokens = array)
    
    #print(new_movies.head(5))
    
    print(new_data[1:5])
    return(new_data)  
    pass

In [30]:
def featurize(data):
    """
    Append a new column to the movies DataFrame with header 'features'.
    Each row will contain a csr_matrix of shape (1, num_features). Each
    entry in this matrix will contain the tf-idf value of the term, as
    defined in class:
    tfidf(i, d) := tf(i, d) / max_k tf(k, d) * log10(N/df(i))
    where:
    i is a term
    d is a document (movie)
    tf(i, d) is the frequency of term i in document d
    max_k tf(k, d) is the maximum frequency of any term in document d
    N is the number of documents (movies)
    df(i) is the number of unique documents containing term i
    Params:
      movies...The movies DataFrame
    Returns:
      A tuple containing:
      - The movies DataFrame, which has been modified to include a column named 'features'.
      - The vocab, a dict from term to int. Make sure the vocab is sorted alphabetically as in a2 (e.g., {'aardvark': 0, 'boy': 1, ...})
   
    """
    ###TODO
    #print(movies[:5]) 
    
    #step 1 -> build a vocab and df(term)
    vocab = {}
    vocab_list = []
    df = {}
    
    for row1 in data['tokens']:
        row = list(set(row1))
        for term in row:
            if term not in vocab.keys():
                vocab.setdefault(term,-1)
             
            if term not in df.keys(): 
                df.setdefault(term,1)
            else :
                df[term] += 1
             
             
    #print('vocab = ', vocab)
    
    vocab_list = sorted(vocab.keys(), key = lambda x:x)
    #print('vocab_list = ', vocab_list)
    
    for i,term in enumerate(vocab_list):
         vocab[term] = i
            
    #        
         
    #print('Sorted vocab = ', sorted(vocab.items()))
    #print('df = ',sorted(df.items(), key=lambda x:x[0]))
    
    # step 2 -> Build a csr_matrix for each row of movies['tokens']
    N = len(data)
    #print('N = ',N)
    
    #[comedy, comedy, comedy, horror]  -> max_k tf(k, d) = 3 
    #[action, comedy,thriller] -> tf(action, d) =1
    # df(i) ->
    #num_features is the total number of unique features across all documents.
    
    csr_array =[]
    
    for row1 in data['tokens']:
        csr_row = []
        csr_col = []
        csr_data = []
        max_k = 0
       
        #print(row1)
        
        if row1 != []:
            max_k = Counter(row1).most_common()[:1][0][1]
            row = list(set(row1))

            #print(max_k)

            #print('removed duplicates =',row)
            for term in row:       
                csr_row.append(0)
                csr_col.append(vocab[term])
                #tfidf(i, d) := tf(i, d) / max_k tf(k, d) * log10(N/df(i))
                tf = Counter(row1)[term]
                #max_k = max_k.most_common()[:1][0][1]
         
                #print('term = %s ---> tf = %d ---> max_k = %d'%(term,tf,max_k))
                tfidf = (tf / max_k) * math.log10(N/df[term])
                csr_data.append(tfidf)
           

         
        X = csr_matrix((csr_data, (csr_row, csr_col)), shape=(1, len(vocab)), dtype=np.float64)
       
        #print('X ->\n',X.toarray())
        csr_array.append(X)
    

    # step 3 -> add column features to movies 
    #print('size of csr_array = ',len(csr_array)) 
    #print('CSR = ',csr_array[:2])  
    new_data = data.assign(features = csr_array)
    #print(new_movies.head(2))
     
    return(new_data,vocab)   
    
    pass

In [31]:
d1 = d[1:100]

In [32]:
#print(d1)

In [33]:
data = tokenize(d1)

['number', 'would', 'onlybwith', 'original', 'company', 'harassing', 'called', 'sending', 'contact', 'messsage', 'would', 'report', 'contacted', 'people', 'whose', 'concern', 'business']
['longtime', 'member', 'charter', 'citizens', 'mother', 'became', 'elderly', 'assisted', 'opening', 'account', 'charter', 'branch', 'office', 'located', 'michigan', 'placed', 'account', 'account', 'owner', 'mother', 'became', 'necessary', 'obtain', 'durable', 'power', 'attorney', 'handle', 'financial', 'medical', 'needs', 'years', 'mother', 'present', 'presented', 'original', 'stamped', 'durable', 'power', 'attorney', 'charter', 'branch', 'office', 'located', 'michigan', 'record', 'presented', 'understanding', 'would', 'placed', 'mother', 'account', 'furthermore', 'needed', 'transact', 'business', 'would', 'welcomed', 'though', 'contacted', 'charter', 'master', 'someone', 'fraudulent', 'charges', 'mother', 'credit', 'account', 'caller', 'stated', 'would', 'immediately', 'close', 'mother', 'master', 'cr

In [34]:
print(data['tokens'][1:5])

56884    [longtime, member, charter, citizens, mother, ...
56894    [looking, credit, report, collection, account,...
56898    [received, stating, wanted, however, would, as...
56910    [contacted, years, later, private, supposedly,...
Name: tokens, dtype: object


In [35]:
data, vocab = featurize(data)

In [36]:
print('vocab:')
print(sorted(vocab.items())[:500])

vocab:
[('ability', 0), ('abysmal', 1), ('accept', 2), ('acceptable', 3), ('acceptance', 4), ('accepted', 5), ('access', 6), ('accommodate', 7), ('accordance', 8), ('according', 9), ('account', 10), ('accounting', 11), ('accounts', 12), ('accpet', 13), ('accurate', 14), ('achieve', 15), ('acount', 16), ('acquired', 17), ('acted', 18), ('action', 19), ('actions', 20), ('actuality', 21), ('actually', 22), ('added', 23), ('addition', 24), ('additional', 25), ('address', 26), ('addresses', 27), ('adjudication', 28), ('adjustable', 29), ('adjustments', 30), ('advantage', 31), ('adversely', 32), ('advertised', 33), ('advertisement', 34), ('advertisers', 35), ('advertising', 36), ('advise', 37), ('advised', 38), ('advising', 39), ('affected', 40), ('afford', 41), ('affordable', 42), ('agencies', 43), ('agency', 44), ('agent', 45), ('agree', 46), ('agreed', 47), ('agreeing', 48), ('agreement', 49), ('ahead', 50), ('airport', 51), ('alert', 52), ('align', 53), ('allegedly', 54), ('allow', 55), 

In [37]:
print(len(vocab))

1686


In [38]:
print(data['features'][1:5])

56884      (0, 1)\t0.166302932883\n  (0, 10)\t0.3205751...
56894      (0, 10)\t0.106858367633\n  (0, 12)\t0.238560...
56898      (0, 110)\t0.336707949932\n  (0, 178)\t0.7592...
56910      (0, 44)\t0.497817597299\n  (0, 75)\t1.518513...
Name: features, dtype: object


In [39]:
def train_test_split(data):
    """DONE.
    Returns a random split of the ratings matrix into a training and testing set.
    """
    test = set(range(len(data))[::10])
    train = sorted(set(range(len(data))) - test)
    test = sorted(test)
    return data.iloc[train], data.iloc[test]

In [40]:
data_train, data_test = train_test_split(data)
print('%d training ratings; %d testing ratings' % (len(data_train), len(data_test)))

89 training ratings; 10 testing ratings


In [63]:
def cosine_sim(a, b):
    """
    Compute the cosine similarity between two 1-d csr_matrices.
    Each matrix represents the tf-idf feature vector of a movie.
    Params:
      a...A csr_matrix with shape (1, number_features)
      b...A csr_matrix with shape (1, number_features)
    Returns:
      The cosine similarity, defined as: dot(a, b) / ||a|| * ||b||
      where ||a|| indicates the Euclidean norm (aka L2 norm) of vector a.
      
    >>> a = csr_matrix([[ 0.0,  0.0,  0.0,  0.0, 0.43974934,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0, 0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  1.3955668,  0.0]] )
    >>> b = csr_matrix([[ 0.0,  0.0,  0.0,  0.0,  0.43974934,  0.0,  0.0,  0.32024863,  0.0, 0.0,  0.0,  0.0, 0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0]] )
    >>> sim = cosine_sim(a, b)
    >>> print(sim)
    0.242942001121
    >>> a =  csr_matrix([[ 0.0,  0.0,        0.0,        0.0,        0.43974934,  0.0,  0.0,  0.0,         0.0, 0.0,  0.0,  0.0,  0.0,        0.0,  0.0,        0.0,  0.0,  0.0,  0.0,  0.0,  1.3955668,  0.0]] )
    >>> b =  csr_matrix([[ 0.0,  0.9121797,  1.3099253,  1.1945643,  0.0,         0.0,  0.0,  0.32024863,  0.0,  0.0,  0.0,  0.0,  1.7755414,  0.0,  1.3647367,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,        0.0]] )
    >>> sim = cosine_sim(a, b)
    >>> print(sim)
    0.0
    """
    ###TODO
    #print(type(a))
    #print(type(b))
    #print('a = ',a)
    #print('b = ',b)
    
    #step 1 -> calculate ||a||
    x = (a.data * a.data)
    X = math.sqrt(x.sum())    
    #print('X = ',X)

    #step 2 -> calculate ||b||
    y = (b.data * b.data)
    Y = math.sqrt(y.sum())    
    #print('Y = ',Y)

    # step 3 -> calculate dot(a, b)
    #print('a = ',a.todense())
    #print('b = ',b.todense())
    #print('a = ',a.shape)
    #print('b = ',b.shape)
    
    
    dotProduct = (a).dot(b.transpose())
    #print('dotProduct = ',dotProduct)

    Sum_dot = dotProduct.sum()
    #print('Sum of dotProduct = ',Sum_dot)
    
    # step 4 -> calculate cosine similarity
    if (X != 0) | (Y !=0) :
        Cos_Sim = Sum_dot / ( X * Y )
    else :
        Cos_Sim = 0
    
    #print('Cos_Sim = ',Cos_Sim)
    return(Cos_Sim)
    
    pass

In [64]:
print(data[1:5])

      Date received           Product               Sub-product  \
56884     3/23/2015       Credit card                       NaN   
56894     3/23/2015  Credit reporting                       NaN   
56898     3/23/2015   Debt collection               Payday loan   
56910     3/23/2015      Student loan  Non-federal student loan   

                                        Issue  \
56884                                 Rewards   
56894  Incorrect information on credit report   
56898     Improper contact or sharing of info   
56910      Dealing with my lender or servicer   

                                   Sub-issue  \
56884                                    NaN   
56894                Information is not mine   
56898  Talked to a third party about my debt   
56910  Having problems with customer service   

                            Consumer complaint narrative  \
56884  I 'm a longtime member of Charter One Bank/RBS...   
56894  After looking at my credit report, I saw a col... 

In [80]:
def make_predictions(data, data_train, data_test):
    """
    Using the ratings in ratings_train, predict the ratings for each
    row in ratings_test.
    To predict the rating of user u for movie i: Compute the weighted average
    rating for every other movie that u has rated.  Restrict this weighted
    average to movies that have a positive cosine similarity with movie
    i. The weight for movie m corresponds to the cosine similarity between m
    and i.
   
    If there are no other movies with positive cosine similarity to use in the
    prediction, use the mean rating of the target user in ratings_train as the
    prediction.
    Params:
      movies..........The movies DataFrame.
      ratings_train...The subset of ratings used for making predictions. These are the "historical" data.
      ratings_test....The subset of ratings that need to predicted. These are the "future" data.
    Returns:
      A numpy array containing one predicted rating for each element of ratings_test.
    """
    ###TODO
    #print('movies = \n',movies.head(2))
    #print('ratings_train = \n',ratings_train.head(2))
    #print('ratings_test = \n',ratings_test.head(2))
    
    # steps ->  traverse each row of ratings_test 
    # find user's movie To predict and his movie list that he rated 
    # and then find cosine sim
    
    
    result = [] 
    wrong_pred = 0
    
    for index, row in data_test.iterrows():
        Complain_ID = row['Complaint ID']
        product = row['Product']
        issue = row['Issue'] 
       
        #user_movieTopredict = movies[(movies.movieId == test_movie)]
        #a =  user_movieTopredict['features'].values[0]
        
        responce_Topredict = data[(data['Complaint ID'] == Complain_ID )]
        a =  responce_Topredict['features'].values[0]

        trains_data = data_train[ (data_train['Complaint ID'] != Complain_ID) & ((data_train.Product == product) | (data_train.Issue == issue))]
        
        print("Test Data")
        #print(Complain_ID)
        #print(product)
        #print(issue)          
        #print(trains_data['Complaint ID'])
    

        update_cos_sim = 0
        higest_cos_match = 0
        
        for index1, row1 in trains_data.iterrows():          
                data_Hist = data[(data['Complaint ID'] == row1['Complaint ID'])]
                print("History Data")
                #print(data_Hist['Product'])
                #print(data_Hist['Complaint ID'])
                b =  data_Hist['features'].values[0]


                cos_sim = cosine_sim(a,b)
                print(cos_sim) 
              
                 
                if (cos_sim > 0) & (update_cos_sim < cos_sim ):
                        update_cos_sim = cos_sim   
                        #print(update_cos_sim)   
                        #print(row1['Product'])
                        print(row1['Complaint ID'])
                        higest_cos_match = row1['Complaint ID']
                        
        print("Cos_Sim",update_cos_sim)            
        print('Complaint_ID=%d test_product=%s \nActual_responce=%s \nPred_responce=%s'%(Complain_ID,product,row['Company response to consumer'],row1['Company response to consumer']))  
        result.append((higest_cos_match,update_cos_sim))
        
        if row['Company response to consumer'] != row1['Company response to consumer'] :
               wrong_pred = wrong_pred + 1
    
    return(result,wrong_pred)    


    pass



In [81]:
predictions,wrong_pred = make_predictions(data, data_train, data_test)

Test Data
History Data
0.0746004522195
1296727.0
History Data
0.0308705035938
History Data
0.0
History Data
0.0467599384091
History Data
0.0513459229021
History Data
0.0495144610638
History Data
0.0158172366563
History Data
0.0251522392525
History Data
0.0659831895746
History Data
0.0254674278349
History Data
0.0775321336975
1297758.0
History Data
0.057168314283
History Data
0.057168314283
History Data
0.0713544945092
History Data
0.0
History Data
0.0
History Data
0.057996345386
History Data
0.0365245841916
History Data
0.0148165183722
History Data
0.0230377163436
History Data
0.0607152217409
History Data
0.0403656638218
History Data
0.0194767279768
History Data
0.0
History Data
0.0192287779328
History Data
0.0129922352007
History Data
0.0171644432945
History Data
0.0257323198435
History Data
0.0
Cos_Sim 0.0775321336975
Complaint_ID=1296593 test_product=Debt collection 
Actual_responce=Closed with explanation 
Pred_responce=Closed with explanation
Test Data
History Data
0.0150816152004

In [82]:
def mean_error(wrong_pred, data_test):
    """DONE.
    Return the mean absolute error of the predictions.
    """
    error = 0.0
    
    error = (wrong_pred / len(data_test) ) * 100
    
    return error


In [83]:
print('error=%f' % mean_error(wrong_pred, data_test))
print(predictions[:10])

error=20.000000
[(1297758.0, 0.077532133697531661), (1297685.0, 0.042515788024741943), (1297977.0, 0.10868431684554017), (1304502.0, 0.17076250595109407), (1304678.0, 0.21947021989073259), (1299258.0, 0.13753025764504803), (1297457.0, 0.11788305445317175), (1290545.0, 0.10705834544488203), (1295228.0, 0.051190210585378537), (1296955.0, 0.16384681213751506)]
